In [1]:
import numpy as np 
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import chardet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from IPython.display import clear_output
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
# Import data from csv and create a database (including table) and a dataframe.

Path('maternity_risk_data.db').touch()
#con = sqlite3.connect("/kaggle/working/maternity_risk_data.db")
con = sqlite3.connect("maternity_risk_data.db")
cur = con.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS risk_tb (Age, SystolicBP, DiastolicBP, BS, BodyTemp, HeartRate, RiskLevel)''')
df = pd.read_csv('Maternal Health Risk Data Set.csv')
index = [i + 1 for i in df.index]
df.insert(1, 'index_col', index) # Add a column for index since it was missing from original dataset. 
df.to_sql('risk_tb', con, if_exists='replace', index=False)
pregnancy_risk = pd.read_sql('''SELECT * FROM risk_tb''',con)
pregnancy_risk

,Age,index_col,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,1,130,80,15.0,98.0,86,high risk
1,35,2,140,90,13.0,98.0,70,high risk
2,29,3,90,70,8.0,100.0,80,high risk
3,30,4,140,85,7.0,98.0,70,high risk
4,35,5,120,60,6.1,98.0,76,low risk
...,...,...,...,...,...,...,...,...
1009,22,1010,120,60,15.0,98.0,80,high risk
1010,55,1011,120,90,18.0,98.0,60,high risk
1011,35,1012,85,60,19.0,98.0,86,high risk
1012,43,1013,120,90,18.0,98.0,70,high risk


In [2]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(pregnancy_risk):
    # Drop column: 'index_col'
    pregnancy_risk = pregnancy_risk.drop(columns=['index_col'])
    return pregnancy_risk

pregnancy_risk_clean = clean_data(pregnancy_risk.copy())
pregnancy_risk_clean.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


In [3]:
# View the first few rows of the DataFrame
print(pregnancy_risk.head())

# Summary statistics for numerical columns
print(pregnancy_risk.describe())

# Information on the DataFrame including the data types of each column
print(pregnancy_risk.info())


   Age  index_col  SystolicBP  DiastolicBP    BS  BodyTemp  HeartRate  \
0   25          1         130           80  15.0      98.0         86   
1   35          2         140           90  13.0      98.0         70   
2   29          3          90           70   8.0     100.0         80   
3   30          4         140           85   7.0      98.0         70   
4   35          5         120           60   6.1      98.0         76   

   RiskLevel  
0  high risk  
1  high risk  
2  high risk  
3  high risk  
4   low risk  
               Age    index_col   SystolicBP  DiastolicBP           BS  \
count  1014.000000  1014.000000  1014.000000  1014.000000  1014.000000   
mean     29.871795   507.500000   113.198225    76.460552     8.725986   
std      13.474386   292.860888    18.403913    13.885796     3.293532   
min      10.000000     1.000000    70.000000    49.000000     6.000000   
25%      19.000000   254.250000   100.000000    65.000000     6.900000   
50%      26.000000   507.50

In [4]:
# Drop column: 'index_col'
pregnancy_risk = pregnancy_risk.drop(columns=['index_col'])

In [5]:
pregnancy_risk.columns

Index(['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate',
       'RiskLevel'],
      dtype='object')

In [6]:
# For numerical columns
for col in pregnancy_risk.select_dtypes(include=['float64', 'int64']):
    pregnancy_risk[col].fillna(pregnancy_risk[col].mean(), inplace=True)

# For categorical columns
for col in pregnancy_risk.select_dtypes(include=['object']):
    pregnancy_risk[col].fillna(pregnancy_risk[col].mode()[0], inplace=True)


ignore this section for now

In [39]:
#dummies = pd.get_dummies(pregnancy_risk)


In [38]:
#print(dummies.columns)


In [44]:
# from sklearn.model_selection import train_test_split

# # Age is the column with the target variable and the rest are features
# X = dummies.drop('Age', axis=1)
# y = dummies['Age']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [47]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


processing data

In [32]:
# One hot encoding by pandas
ohe=pd.get_dummies(pregnancy_risk, drop_first=True)

In [33]:
# Display first few lines
ohe.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel_high risk,RiskLevel_low risk,RiskLevel_mid risk
0,25,130,80,15.0,98.0,86,1,0,0
1,35,140,90,13.0,98.0,70,1,0,0
2,29,90,70,8.0,100.0,80,1,0,0
3,30,140,85,7.0,98.0,70,1,0,0
4,35,120,60,6.1,98.0,76,0,1,0


In [40]:
print(ohe.columns)

Index(['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate',
       'RiskLevel_high risk', 'RiskLevel_low risk', 'RiskLevel_mid risk'],
      dtype='object')


In [66]:
# X=ohe.drop('Age',axis=1)
# y=ohe['Age']

In [67]:
# Set age as feature
X=ohe.drop('RiskLevel_high risk',axis=1)
y=ohe['RiskLevel_high risk']

In [68]:
#standardize
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the features
X = scaler.fit_transform(X)



In [69]:
# Train modeling
from sklearn.model_selection import train_test_split

# Split the data, let's use 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)



LogisticRegression(max_iter=10000)

In [71]:
# Make predictions
y_pred = model.predict(X_test)


In [72]:
# Print classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       156
           1       1.00      1.00      1.00        47

    accuracy                           1.00       203
   macro avg       1.00      1.00      1.00       203
weighted avg       1.00      1.00      1.00       203



Playing with other options

In [57]:
# Initialize the model
model2= LogisticRegression(solver='newton-cg')


# Train the model
model2.fit(X_train, y_train)


LogisticRegression(solver='newton-cg')